In [2]:
# !pip install bs4
#!pip install lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [10]:
# URL de la página web que quieres scrapear
url = "www.example.com"

def obtener_soup(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Crear un objeto BeautifulSoup con el contenido de la página
        soup = BeautifulSoup(response.text, 'html.parser')
    return soup

soup = obtener_soup(url)

In [12]:
def obtener_primeras_imagenes(urls):
    # Mantener solo la primera imagen de cada grupo
    primeras_imagenes = []
    imagen_anterior = ""

    for url in urls:        
        id_imagen = url.split('/')[-1]  # Extrae el ID del producto de la URL
        id_imagen = id_imagen.split('-')[0].split('.')[0].split('_')[0]

       # print(url,id_imagen,imagen_anterior,id_imagen[0],id_imagen[0].isdigit())
        if id_imagen != imagen_anterior and id_imagen[0].isdigit(): #and id_imagen[0].isdigit():# :  
            
            primeras_imagenes.append(url)
            imagen_anterior = id_imagen

    return primeras_imagenes


def agregar_p(url_base,urls):

    nuevos_links = []
    for url in urls:
        url_nueva = url_base + url 
        nuevos_links.append(url_nueva)
    return nuevos_links

In [24]:
def Scrapper(soup,url_base= "www.example.com/"):
    key_to_search = ["productReference","productName","linkText","Precios"]#"highPrice"
    key_multiples = ["imageUrl"]

    lista_variables = []
    largo_listas = []

    
    for ky in key_to_search:
        
        pattern = rf'"{ky}":"(.*?)"'       
        
        results = re.findall(pattern, soup)
        link_text_list = list(results)
         
        if ky in key_multiples:
            link_text_list = obtener_primeras_imagenes(link_text_list)

        if ky == 'linkText': 
            link_text_list = agregar_p(url_base,link_text_list)
            
        lista_variables.append(link_text_list)
        largo_listas.append(len(link_text_list))
    data = {
        'SKU': lista_variables[0],
        'NOMBRE': lista_variables[1],
        'LINK': lista_variables[2],
        'PRECIO': lista_variables[3]}

    df = pd.DataFrame(data)
    return df
Scrapper(soup,url_base= "www.example.com/").head(6)

,SKU,NOMBRE,LINK,PRECIO
0,3513562,Sofá New Alma 3 Puestos Taupe,www.example.com/sofa-new-alma-3-puestos-taupe-,3199990
1,3513561,Sofá New Alma 3 Puestos Humo,www.example.com/sofa-new-alma-3-puestos-humo-,3199990
2,3512881,Sofá New Alma 2 Puestos Humo,www.example.com/sofa-new-alma-2-puestos-humo-,2799990
3,3512882,Sofá New Alma 2 Puestos Taupe,www.example.com/sofa-new-alma-2-puestos-taupe-,2799990
4,3513560,Sofá New Alma 3 Puestos Azul,www.example.com/sofa-new-alma-3-puestos-azul-,2799990
5,3512880,Sofá New Alma 2 Puestos Azul,www.example.com/sofa-new-alma-2-puestos-azul-,2799990
